# 02 - Generating Dataset 1

The purpose of this notebook is to create a dataset that will be used for testing the performance of the package and conducting a series of experiments.

By separating the creation of the dataset from its use it should be easier to re-use the dataset for the different tests and experiments.

In [ ]:
import json
import os

In [ ]:
from privacy_fingerprint.common.config import (
    load_global_config_from_file,
    load_experiment_config_from_file,
    load_experiment_config,
)
import privacy_fingerprint.generate.synthea as synthea
import privacy_fingerprint.generate.language_model as llm
import privacy_fingerprint.extract.aws_comprehend as aws

In [ ]:
# Example config files are available in the config directory.
# These files will need to be customised with your API keys.

load_global_config_from_file("../configs/global_config.yaml")
load_experiment_config_from_file("../configs/experiment_config.yaml")

# Config options can be modified inline. To keep this notebook/experiment small
# the number of records will be changed to 10.
expt_config = load_experiment_config()
expt_config.synthea.encounter_type = "Encounter for symptom"
expt_config.synthea.num_records = 10  # 100_000 used to create dataset1
load_experiment_config(expt_config.dict())

In [ ]:
# The Synthea output will be saved to a directory
output_dir = "../experiments/02_generate_dataset"
os.makedirs(output_dir, exist_ok=True)
export_directory = os.path.join(output_dir, "synthea")

In [ ]:
# CAUTION: Given the number of records, running this cell will be extremely slow.

# Generate structured records
synthea_records = synthea.generate_records(export_directory)

with open(os.path.join(output_dir, "synthea_dataset.json"), "w") as fp:
    json.dump(synthea_records, fp)

A modified version of the above was run. This generated 100k records in Synthea but then limited the import of those records to 1000. This then formed our dataset1. The records generated in our run are available separately to this repository.

In [ ]:
# If using a previously generated set of records they can be loaded as follows:

with open(os.path.join(output_dir, "synthea_dataset.json")) as fp:
    synthea_records = json.load(fp)

The structured notes from Synthea can then be converted to free-text clinical notes.

In [ ]:
clinical_note_generator = llm.LMGenerator()
llm_results = list(clinical_note_generator.generate_text(synthea_records))

with open(os.path.join(output_dir, "llm_dataset.json"), "w") as fp:
    json.dump(llm_results, fp)

In [ ]:
# If using a previously generated set of records they can be loaded as follows:

with open(os.path.join(output_dir, "llm_dataset.json")) as fp:
    llm_results = json.load(fp)

In [ ]:
# The NER step using AWS ComprehendMedical is the most expensive step.
# The cost can be estimated with the following function:

print("Estimated cost is $", aws.calculate_ner_cost(llm_results))

In [ ]:
aws_extract = aws.ComprehendExtractor()
ner_records = [aws_extract.extract_record(r) for r in llm_results]

with open(os.path.join(output_dir, "ner_dataset.json"), "w") as fp:
    json.dump(ner_records, fp)

In [ ]:
# If using a previously generated set of records they can be loaded as follows:

with open(os.path.join(output_dir, "ner_dataset.json")) as fp:
    ner_records = json.load(fp)

With the raw NER results generated, experiments will move to individual notebooks.